In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from __future__ import print_function
import keras
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Activation, Concatenate, \
                                    AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import numpy as np
import os
import time
import pandas as pd
from numpy.linalg import inv, norm
from tqdm import tqdm
from keras.preprocessing import image

np.set_printoptions(suppress=True, precision=3)

save_dir = os.path.join(os.getcwd(), 'saved_models')

In [ ]:
train_labels = pd.read_csv('/content/gdrive/MyDrive/Xray dataset/labels_train.csv')
test_labels = pd.read_csv('/content/gdrive/MyDrive/Xray dataset/sample_submission.csv')
dataset_url = '/content/gdrive/MyDrive/Xray dataset'
directory_train = '/content/gdrive/MyDrive/Xray dataset/train_images/'
directory_test = '/content/gdrive/MyDrive/Xray dataset/test_images/'
trainingNPY = '/content/gdrive/MyDrive/pneumonia/training.npy'
testingNPY = '/content/gdrive/MyDrive/pneumonia/testing.npy'

In [ ]:
x_train=np.load(trainingNPY)
x_test=np.load(testingNPY)

In [ ]:
IMG_SIZE = 128
x_train = np.array(x_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
x_test = np.array(x_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
y_train = np.array(train_labels.drop(['file_name'],axis=1))
y_train.shape
print(y_train)
y_test = np.array(test_labels.drop(['file_name'],axis=1))
y_test.shape
print(len(y_test))

[[1]
 [2]
 [2]
 ...
 [1]
 [1]
 [1]]
1168


In [ ]:
num_classes=3

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

t_train = keras.utils.to_categorical(y_train, num_classes)
t_test = keras.utils.to_categorical(y_test, num_classes)

print('y_train (labels) shape:', y_train.shape)
print('t_train (one-hot rep) shape:', t_train.shape)

y_train (labels) shape: (4672, 1)
t_train (one-hot rep) shape: (4672, 3)


In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D


model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, t_train, batch_size=32, epochs=10, validation_split=0.01)

Epoch 1/10
145/145 [==============================] - 15s 54ms/step - loss: 1.0707 - accuracy: 0.6360 - val_loss: 0.5409 - val_accuracy: 0.7234
Epoch 2/10
145/145 [==============================] - 7s 49ms/step - loss: 0.4851 - accuracy: 0.7954 - val_loss: 0.5498 - val_accuracy: 0.7872
Epoch 3/10
145/145 [==============================] - 7s 49ms/step - loss: 0.4407 - accuracy: 0.8116 - val_loss: 0.5111 - val_accuracy: 0.7660
Epoch 4/10
145/145 [==============================] - 7s 49ms/step - loss: 0.4173 - accuracy: 0.8230 - val_loss: 0.4972 - val_accuracy: 0.7234
Epoch 5/10
145/145 [==============================] - 7s 49ms/step - loss: 0.4125 - accuracy: 0.8236 - val_loss: 0.5383 - val_accuracy: 0.7447
Epoch 6/10
145/145 [==============================] - 7s 49ms/step - loss: 0.3428 - accuracy: 0.8566 - val_loss: 0.5541 - val_accuracy: 0.6809
Epoch 7/10
145/145 [==============================] - 7s 49ms/step - loss: 0.3006 - accuracy: 0.8818 - val_loss: 0.4332 - val_accuracy: 0.787

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 256)     2560      
_________________________________________________________________
activation (Activation)      (None, 126, 126, 256)     0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 256)       590080    
_________________________________________________________________
activation_1 (Activation)    (None, 61, 61, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 230400)            0

In [ ]:
model.save('simplestSequential.model')

INFO:tensorflow:Assets written to: simplestSequential.model/assets


In [ ]:
model = tf.keras.models.load_model("simplestSequential.model")
prediction = model.predict(x_test)

In [ ]:
with np.printoptions(threshold=np.inf):
    print(prediction)

[[1.    0.    0.   ]
 [1.    0.    0.   ]
 [0.    1.    0.   ]
 [0.    0.056 0.944]
 [0.    0.435 0.565]
 [0.    0.044 0.956]
 [0.    0.979 0.021]
 [0.    0.977 0.023]
 [0.    0.986 0.014]
 [1.    0.    0.   ]
 [0.988 0.012 0.   ]
 [0.    0.492 0.508]
 [0.002 0.169 0.829]
 [0.007 0.424 0.569]
 [0.    0.975 0.025]
 [1.    0.    0.   ]
 [0.345 0.341 0.314]
 [0.    0.003 0.997]
 [0.    0.995 0.005]
 [0.011 0.988 0.001]
 [0.    0.945 0.055]
 [1.    0.    0.   ]
 [0.    0.811 0.189]
 [1.    0.    0.   ]
 [0.    0.962 0.038]
 [0.    1.    0.   ]
 [1.    0.    0.   ]
 [1.    0.    0.   ]
 [0.    0.991 0.009]
 [1.    0.    0.   ]
 [0.003 0.856 0.14 ]
 [1.    0.    0.   ]
 [1.    0.    0.   ]
 [1.    0.    0.   ]
 [0.    0.998 0.002]
 [1.    0.    0.   ]
 [0.001 0.99  0.009]
 [0.999 0.    0.001]
 [0.693 0.301 0.006]
 [0.001 0.967 0.032]
 [1.    0.    0.   ]
 [0.    0.783 0.217]
 [0.    0.041 0.959]
 [1.    0.    0.   ]
 [1.    0.    0.   ]
 [0.    0.245 0.755]
 [1.    0.    0.   ]
 [0.    0.989

In [ ]:
label_pred = np.array([np.argmax(prediction, axis=1)])

In [ ]:
print(label_pred.T)

[[0]
 [0]
 [1]
 ...
 [2]
 [1]
 [1]]


In [ ]:
predict_labels = np.array(test_labels.drop(['class_id'],axis=1))
predict_labels = np.append(predict_labels,label_pred.T,axis=1)
with np.printoptions(threshold=np.inf):
    print(predict_labels)

[['img_6309102362127931879.jpg' 0]
 ['img_2031896927656989520.jpg' 0]
 ['img_4693592482016108254.jpg' 1]
 ['img_1731353163814999872.jpg' 2]
 ['img_4270405616056570211.jpg' 2]
 ['img_5592512817605259347.jpg' 2]
 ['img_7270920511344208641.jpg' 1]
 ['img_8750976362013359084.jpg' 1]
 ['img_3099813306519204140.jpg' 1]
 ['img_6152245361589836720.jpg' 0]
 ['img_5133964404089301517.jpg' 0]
 ['img_6094498399694976619.jpg' 2]
 ['img_5009290568007555183.jpg' 2]
 ['img_91583952582106774.jpg' 2]
 ['img_6172231491103496150.jpg' 1]
 ['img_7279243678066448832.jpg' 0]
 ['img_1508041442905435888.jpg' 0]
 ['img_5834118063979502391.jpg' 2]
 ['img_6587768937848096651.jpg' 1]
 ['img_3077954441619156589.jpg' 1]
 ['img_3655945510172717944.jpg' 1]
 ['img_3153198158239991036.jpg' 0]
 ['img_8068040535899796305.jpg' 1]
 ['img_785206936656049715.jpg' 0]
 ['img_4635476933370359163.jpg' 1]
 ['img_7081409369711501166.jpg' 1]
 ['img_8166760919851749947.jpg' 0]
 ['img_8669652425730035268.jpg' 0]
 ['img_7169925662333297

In [ ]:
df_for_export = pd.DataFrame(predict_labels,columns=['file_name', 'class_id'])

df_for_export.to_csv('df_for_export.csv')

In [ ]:
print(df_for_export)

                        file_name class_id
0     img_6309102362127931879.jpg        0
1     img_2031896927656989520.jpg        0
2     img_4693592482016108254.jpg        1
3     img_1731353163814999872.jpg        2
4     img_4270405616056570211.jpg        2
...                           ...      ...
1163  img_3751188215019143871.jpg        0
1164  img_2990304631921044895.jpg        1
1165  img_4362101259165710930.jpg        2
1166   img_972668160331506727.jpg        1
1167  img_7789226562806226056.jpg        1

[1168 rows x 2 columns]


In [ ]:
!pwd

/content
